In [1]:
import pandas as pd
from MVALM.datasets import VGGSound
from pathlib import Path
from MVALM.datasets.VGGSound.vggsound import VGGSoundAudio

In [30]:
df_cls = pd.read_parquet('/home/t9s9/Datasets/VGGSound/annot_test.parquet')
df_full = pd.read_parquet('/home/t9s9/Datasets/VGGSound/annot_full_test.parquet')

In [35]:
pd.read_parquet('/home/t9s9/Datasets/VGGSound/annot_train.parquet')

,tags,filename,target,captions_visual,captions_auditory
0,[waterfall burbling],train/--0PQM4-hqg_000030_000040.mp4,299,[A river filled with lots of water next to a f...,[Water is rushing down a waterfall at a consta...
1,[playing tennis],train/--56QUhyDQM_000185_000195.mp4,224,[The video shows a young woman playing tennis ...,[A tennis ball is being hit on the court.]
2,[people belly laughing],train/--5OkAjCI7g_000040_000050.mp4,140,[Two young boys brushing their teeth in front ...,[A baby is laughing and a woman is talking.]
3,[car engine starting],train/--8puiAGLhs_000030_000040.mp4,29,[How to change the speed of your car.],[Someone is starting a car and then starting i...
4,[alarm clock ringing],train/--96EN9NUQM_000242_000252.mp4,4,[A hand holding a small clock on a table.],[A musical instrument is being played with a c...
...,...,...,...,...,...
164413,[lathe spinning],train/zzsUhaDSqzI_000115_000125.mp4,111,[A factory machine that is making a machine.],[A motor is running at a constant speed.]
164414,[lathe spinning],train/zzsUhaDSqzI_000145_000155.mp4,111,"[In this video, person can be seen working on ...",[A machine is running at a constant speed.]
164415,[people shuffling],train/zztvx3WUBss_000030_000040.mp4,161,[The video shows a young boy jumping over a wa...,[A girl is laughing and playing with a bunch o...
164416,[chicken clucking],train/zzvSVusPPgM_000030_000040.mp4,41,[Biological species in the jungle.],[Several chickens are clucking and cackling in...


In [31]:
df = df_cls.merge(df_full, on='filename', how='inner', validate='one_to_one')
df = df.rename(columns={'label': 'tags'})
df['tags'] = df['tags'].astype('str').apply(lambda x: [x])

In [34]:
df.to_parquet('/home/t9s9/Datasets/VGGSound/annot_test.parquet', index=False)

In [10]:
df = pd.read_parquet('/home/t9s9/Datasets/VGGSound/annot_test.parquet')
df.filename = df.filename.apply(lambda x: str(Path(x).with_suffix('.flac')))
df

,label,filename,target
0,people marching,test/---g-f_I2yQ_000001_000011.flac,157
1,people coughing,test/--U7joUcTCo_000000_000010.flac,145
2,female singing,test/--i-y1v8Hy8_000000_000010.flac,82
3,people belly laughing,test/-0BIyqJj9ZU_000030_000040.flac,140
4,horse neighing,test/-0jeONf82dE_000021_000031.flac,108
...,...,...,...
13620,chimpanzee pant-hooting,test/zz0fdUGTHWo_000045_000055.flac,46
13621,popping popcorn,test/zzFdhaiG_Hk_000095_000105.flac,240
13622,wind noise,test/zzbTaK7CXJY_000030_000040.flac,302
13623,train horning,test/zzqrZOq928w_000055_000065.flac,287


In [14]:
exists = df.filename.apply(lambda x: Path('/home/t9s9/Datasets/VGGSound/audio').joinpath(x).exists())
df[exists].to_parquet('/home/t9s9/Datasets/VGGSound/annot_audio_test.parquet', index=False)

In [4]:
dataset = VGGSoundAudio(root='/home/t9s9/Datasets/VGGSound/', split='test')

In [58]:
df = pd.read_parquet('/home/t9s9/Datasets/VGGSound/annot_train.parquet')
df.label = df.label.astype('category')
idx2label = df.label.cat.categories.to_series().reset_index(drop=True).rename('label').to_frame()
idx2label.to_parquet('/home/t9s9/Datasets/VGGSound/idx_2_label.parquet', index=True)
label2idx = {v: k for k, v in idx2label.to_dict()['label'].items()}

In [59]:
df = df.rename(columns={'target': 'label'})
df['target'] = df.label.cat.codes
df.to_parquet('/home/t9s9/Datasets/VGGSound/annot_train.parquet', index=False)

AttributeError: 'DataFrame' object has no attribute 'cat'

In [61]:
df_test = pd.read_parquet('/home/t9s9/Datasets/VGGSound/annot_test.parquet')
df_test = df_test.rename(columns={'target': 'label'})
df_test['target'] = df_test.label.apply(lambda x: label2idx[x])
df_test.to_parquet('/home/t9s9/Datasets/VGGSound/annot_test.parquet', index=False)
df_test

,label,filename,target
0,people marching,test/---g-f_I2yQ_000001_000011.mp4,157
1,people coughing,test/--U7joUcTCo_000000_000010.mp4,145
2,female singing,test/--i-y1v8Hy8_000000_000010.mp4,82
3,people belly laughing,test/-0BIyqJj9ZU_000030_000040.mp4,140
4,horse neighing,test/-0jeONf82dE_000021_000031.mp4,108
...,...,...,...
13620,chimpanzee pant-hooting,test/zz0fdUGTHWo_000045_000055.mp4,46
13621,popping popcorn,test/zzFdhaiG_Hk_000095_000105.mp4,240
13622,wind noise,test/zzbTaK7CXJY_000030_000040.mp4,302
13623,train horning,test/zzqrZOq928w_000055_000065.mp4,287


In [4]:
dataset = VGGSound(split='test', root='/home/t9s9/Datasets/VGGSound/')
dataset

VGGSound(split=test, len=13625, label=classification)

In [6]:
list(dataset._load_label_names().values())

['air conditioning noise',
 'air horn',
 'airplane',
 'airplane flyby',
 'alarm clock ringing',
 'alligators, crocodiles hissing',
 'ambulance siren',
 'arc welding',
 'baby babbling',
 'baby crying',
 'baby laughter',
 'baltimore oriole calling',
 'barn swallow calling',
 'basketball bounce',
 'bathroom ventilation fan running',
 'beat boxing',
 'bee, wasp, etc. buzzing',
 'bird chirping, tweeting',
 'bird squawking',
 'bird wings flapping',
 'black capped chickadee calling',
 'blowtorch igniting',
 'bouncing on trampoline',
 'bowling impact',
 'bull bellowing',
 'canary calling',
 'cap gun shooting',
 'car engine idling',
 'car engine knocking',
 'car engine starting',
 'car passing by',
 'cat caterwauling',
 'cat growling',
 'cat hissing',
 'cat meowing',
 'cat purring',
 'cattle mooing',
 'cattle, bovinae cowbell',
 'cell phone buzzing',
 'chainsawing trees',
 'cheetah chirrup',
 'chicken clucking',
 'chicken crowing',
 'child singing',
 'child speech, kid speaking',
 'children sho

In [65]:
dataset[0]

VideoOutput(video=tensor([[[[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],

         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],

         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],

         ...,

         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],

         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],

         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   